In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn import set_config

In [119]:
data=pd.read_csv(r"D:\AI & DS\Machine Learning\Weather Data.csv")

In [120]:
data.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [121]:
data.shape

(8784, 8)

In [122]:
data.drop("Date/Time",axis=1,inplace=True)

In [335]:
data.sample(5)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
5911,16.3,12.2,77,7,48.3,101.69,2
1910,21.5,11.8,54,11,24.1,101.83,2
357,-16.9,-22.2,63,15,25.0,103.02,2
6669,14.4,13.7,96,6,9.7,101.07,5
402,-3.5,-5.8,84,22,2.4,99.38,4


In [124]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Temp_C            8784 non-null   float64
 1   Dew Point Temp_C  8784 non-null   float64
 2   Rel Hum_%         8784 non-null   int64  
 3   Wind Speed_km/h   8784 non-null   int64  
 4   Visibility_km     8784 non-null   float64
 5   Press_kPa         8784 non-null   float64
 6   Weather           8784 non-null   object 
dtypes: float64(4), int64(2), object(1)
memory usage: 480.5+ KB


In [125]:
data.isna().sum()

Temp_C              0
Dew Point Temp_C    0
Rel Hum_%           0
Wind Speed_km/h     0
Visibility_km       0
Press_kPa           0
Weather             0
dtype: int64

In [126]:
data['Weather'].value_counts()

Weather
Mainly Clear                               2106
Mostly Cloudy                              2069
Cloudy                                     1728
Clear                                      1326
Snow                                        390
Rain                                        306
Rain Showers                                188
Fog                                         150
Rain,Fog                                    116
Drizzle,Fog                                  80
Snow Showers                                 60
Drizzle                                      41
Snow,Fog                                     37
Snow,Blowing Snow                            19
Rain,Snow                                    18
Thunderstorms,Rain Showers                   16
Haze                                         16
Drizzle,Snow,Fog                             15
Freezing Rain                                14
Freezing Drizzle,Snow                        11
Freezing Drizzle                

In [135]:
data['Weather']=data["Weather"].str.lower()

In [136]:
data.Weather=data.Weather.apply(lambda x: 'cloudy' if 'cloudy' in x else x)

In [137]:
data.Weather=data.Weather.apply(lambda x: 'clear' if 'clear' in x else x)

In [138]:
data.Weather=data.Weather.apply(lambda x: 'snow' if 'snow' in x else x)

In [139]:
data.Weather=data.Weather.apply(lambda x: 'rain' if 'rain' in x else x)

In [140]:
data.Weather=data.Weather.apply(lambda x: 'fog' if 'fog' in x else x)

In [141]:
data.Weather=data.Weather.apply(lambda x: 'others' if x in  ['drizzle','haze','freezing drizzle','freezing drizzle','haze','thunderstorms','freezing drizzle,haze'] else x)

In [142]:
data["Weather"].value_counts()

Weather
cloudy    3797
clear     3432
rain       664
snow       581
fog        241
others      69
Name: count, dtype: int64

In [143]:
data.shape

(8784, 7)

In [165]:
data.Weather=data.Weather.apply(lambda x:1 if x=="cloudy" else 2 if x=="clear"  else 3 if x=="rain" else 4 if x=="snow" else 5 if x=="fog" else 6)

In [171]:
X_train,X_test,y_train,y_test=train_test_split(data.drop(columns=['Weather']),data['Weather'],test_size=0.2,random_state=45)

In [172]:
trf1=ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,5))   # there are 7 rows from which two is drop and 5 encoded column added that why 0 to 10 will slice
])

In [306]:
trf2=SelectKBest(score_func=chi2,k=8)

In [307]:
trf3=DecisionTreeClassifier(max_depth=9)

In [308]:
pipe=make_pipeline(trf1,trf2,trf3)

In [309]:
pipe.fit(X_train,y_train)

C:\Users\MS COMPUTER\AppData\Local\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=8 is greater than n_features=5. All the features will be returned.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 5, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000002667C87C900>)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(max_depth=9))])

In [310]:
y_pred=pipe.predict(X_train)

In [311]:
y_predict_test=pipe.predict(X_test)

In [312]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_pred)

0.7082681087234951

In [313]:
accuracy_score(y_test,y_predict_test)

0.5998861696072851

In [314]:
import pickle
pickle.dump(pipe,open('pipe2.pkl','wb'))

In [315]:
import numpy as np

In [316]:
pipe = pickle.load(open('pipe2.pkl','rb'))


In [336]:
test_input2 = np.array([14.4,13.7,96,6	9.7	101.07	],dtype=object).reshape(1,6)

In [337]:
import warnings
warnings.filterwarnings('ignore')

In [338]:
pipe.predict(test_input2)
if pipe.predict(test_input2)==1:
    print("Weather is Cloudy")

elif pipe.predict(test_input2)==2:
    print("Weather is Clear")

elif pipe.predict(test_input2)==3:
    print("Weather is Rain")

elif pipe.predict(test_input2)==4:
    print("Weather is Snow")

elif pipe.predict(test_input2)==5:
    print("Weather is Fog")

else:
    print("Weather is Others")
    
    


Weather is Fog
